<h1 style="text-align:center;">Retrieving iLO License Key for HPE iLO Systems</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook provides an example of how to obtain the license key for HPE iLO (Integrated Lights-Out) systems. The script utilizes the Redfish API and establishes a connection with the HPE server to retrieve the license key information.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. The purpose of the get_license_key function is to retrieve the license key information for HPE iLO (Integrated Lights-Out) systems using the Redfish API.</h3><br>
  Here's a step-by-step breakdown of what the function does:
<br><ul>
<li>Initialization: The function initializes variables license_uri and license_data to store the license key URI and data, respectively.
</li><br><li>Resource Directory Check: It checks if the resource directory is disabled or not available. If so, it follows a specific set of steps to locate the license key URI.
</li><br><li>License Key Retrieval: If the resource directory is not used, the function retrieves the license key URI and data by making appropriate API requests to the HPE server.
</li><br><li>Resource Directory Usage: If the resource directory is available, the function utilizes it to find the relevant URI for the license key.
</li><br><li>Output: If the license data is obtained successfully, the function prints the license information, including the confirmation request or the license key itself, depending on the available data.
</li></ul><br>Overall, the get_license_key function encapsulates the logic to obtain the license key for HPE iLO systems, considering different scenarios such as the presence or absence of the resource directory.
 </div>


In [ ]:
def get_license_key(_redfishobj):
    license_uri = None
    license_data = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        hp_hpe = next(iter(managers_members_response.obj.Oem))
        license_collection_uri = managers_members_response.obj.Oem[hp_hpe].Links['LicenseService']\
                                                                                    ['@odata.id']
        license_collection_response = _redfishobj.get(license_collection_uri)
        license_collection_members_uri = next(iter(license_collection_response.obj\
                                                                        ['Members']))['@odata.id']
        license_data = _redfishobj.get(license_collection_members_uri).dict
        license_uri = _redfishobj.get(license_collection_members_uri).dict['@odata.id']
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#HpeiLOLicense.' in instance['@odata.type']:
                license_uri = instance['@odata.id']
                license_data = _redfishobj.get(license_uri).dict
                break

    if license_data:
        try:
            sys.stdout.write("\n\'License Info\': \'%s\'\n" % \
            json.dumps(license_data['ConfirmationRequest']['EON'], indent=4, sort_keys=True))
        except KeyError:
            sys.stdout.write("\n\'License Info\': \'%s\'\n" % \
            json.dumps(license_data['LicenseKey'], indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is the Main function overview.</h3>
    <br>
 <ul><li>Sets up the configuration parameters for the HPE iLO system, including the system URL, login account, and password.
</li><br><li>Attempts to create a Redfish client object using the specified configuration parameters to establish a connection with the iLO system.
</li><br><li>If the connection is successful, the script proceeds to execute the get_license_key function to retrieve the iLO license key.
</li><br><li>After retrieving the license key, the script logs out from the Redfish client by calling the logout function to end the session and release resources.
</li><br><li>If the connection to the iLO system fails or the server is unreachable, an error message is displayed, and the script exits.
</li></ul><br>In summary, the main function establishes a connection with the HPE server, retrieves ESKM data, and ensures a clean logout from the server.

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b> use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br><br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
   
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    get_license_key(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>